In [1]:
%load_ext tikzmagic
%matplotlib inline

# 가중치 검정

앞 절에서 선형회귀분석을 사용하여 계산한 가중치의 추정값 $\hat{w}$은 자연법칙에서 정해진 정확한 가중치 $w$와 다를 수 있다고 하였다. 팁 데이터를 사용하여 이 사실을 좀 더 자세히 설명한다.

우리는 현재 다음과 같은 244개 레코드의 팁 데이터를 가지고 있다.

In [3]:
import seaborn as sns

tips = sns.load_dataset("tips")
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


하지만 이 팁 데이터는 특정한 기간동안 식당에서 수집한 데이터일 것이다. 이 데이터를 수집하기 전에도 데이터는 계속 나오고 있었고 수집과정이 끝난 이후에도 데이터는 계속 생성되어 나올 것이다.
만약 우리가 선형회귀분석으로 구한 가중치가 자연법칙에 의해 정해진 정확한 값이라면 이 데이터 뿐 아니라 다른 시점에 나온 데이터를 쓴다고 하더라고 같은 가중치가 나와야 할 것이다. 
같은 논리로 우리가 가진 244개의 데이터 중 일부만 사용하여 가중치를 계산한다고 하더라고 항상 같은 값이 나와야 한다. 이 사실을 검증하기 위해 244개의 데이터를 122개씩 2개의 데이터셋으로 나누어 각각 가중치를 구해보자

In [4]:
tips1 = tips.iloc[:122]
tips2 = tips.iloc[122:]

앞부분 절반의 데이터만 사용하여 가중치를 구하면 다음과 같이 나온다.

In [8]:
import statsmodels.api as sm

result1 = sm.OLS.from_formula("tip ~ scale(total_bill) + scale(size) + C(sex)", tips1).fit()
result1.params

Intercept            3.090691
C(sex)[T.Female]    -0.116007
scale(total_bill)    0.877198
scale(size)          0.061576
dtype: float64

이번에는 뒷부분 절반의 데이터만 사용하여 가중치를 구한 결과이다.

In [10]:
result2 = sm.OLS.from_formula("tip ~ scale(total_bill) + scale(size) + C(sex)", tips2).fit()
result2.params

Intercept            2.878103
C(sex)[T.Female]     0.164822
scale(total_bill)    0.788890
scale(size)          0.300813
dtype: float64

두 선형회귀분석 결과가 서로 다르다는 것을 알 수 있다. 심지어 sex 데이터의 가중치는 부호조차 서로 다르다.